In [1]:
from torch import nn, sqrt
import torch
import sys
from math import sqrt

import sys
sys.path.append('../')

from coatnet import *

In [2]:
import easydict 
args = easydict.EasyDict({ "batch_size": 32, 
                          "epochs": 30, 
                          "data": 0, 
                          'lr':0.01,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':0,
                          'workers':32,
                         'print_freq':2000,
                         'saved_dir':'../trained_models/checkpoint.pt'})

In [3]:
# x=torch.randn(2,3,224,224)
# model=CoAtNet(3,224)
# out = model(x)
# print(out.shape)

In [4]:
# Data loading code
import os
import torchvision.transforms as transforms
import torchvision.datasets as datasets

path = '../ILSVRC/Data/CLS-LOC/'
traindir = os.path.join(path, 'train')
valdir = os.path.join(path, 'val')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))

In [5]:
# train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
train_sampler = None
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, shuffle=(train_sampler is None),
    num_workers=4, pin_memory=False, sampler=train_sampler)

In [6]:
# (train_sampler is None)

In [7]:
images, labels = next(iter(train_loader))
images = images.cuda()
labels = labels.cuda()

In [8]:
model=coatnet_0().cuda()
out = model(images)

/home/beomgon/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272068694/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


initialized


In [9]:
labels

tensor([531, 555, 601, 736, 677,  33, 512, 657, 917, 501, 790, 173, 312, 475,
        815, 674, 315, 754, 570, 807, 296, 529,  44, 622, 570, 486, 960, 784,
        328, 466, 746, 414], device='cuda:0')

In [10]:
print(out.shape)

torch.Size([32, 1000])


In [11]:
torch.argmax(out, dim = -1)

tensor([867, 867, 867, 867, 883, 867, 867, 867, 867, 820, 293, 867, 867, 867,
        867, 867, 883, 867, 867, 883, 867, 867, 867, 867, 867, 867, 867, 867,
        359, 867, 867, 867], device='cuda:0')

In [12]:
args.weight_decay

0.0001

In [13]:
# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().cuda()

optimizer = torch.optim.SGD(model.parameters(), args.lr,
                            momentum=args.momentum,
                            weight_decay=args.weight_decay)
# optimizer = torch.optim.SGD(model.parameters(), 0.1)

In [14]:
output = model(images)
loss = criterion(output, labels)

In [15]:
loss

tensor(9.6655, device='cuda:0', grad_fn=<NllLossBackward0>)

In [16]:
# model.s4.fc_q.weight.shape

In [17]:
# model.s4.fc_q.weight[0, :10]
param = model.fc.weight.detach().clone()
param

tensor([[ 0.0044, -0.0150,  0.0119,  ...,  0.0085, -0.0020,  0.0304],
        [-0.0327,  0.0088,  0.0080,  ..., -0.0007, -0.0302, -0.0132],
        [ 0.0263, -0.0051, -0.0282,  ...,  0.0054, -0.0325,  0.0004],
        ...,
        [ 0.0047,  0.0240,  0.0179,  ...,  0.0115,  0.0091,  0.0306],
        [-0.0266, -0.0221, -0.0271,  ..., -0.0103, -0.0120, -0.0068],
        [ 0.0159,  0.0119, -0.0166,  ...,  0.0236,  0.0216,  0.0035]],
       device='cuda:0')

In [18]:
# loss.backward()k

In [19]:
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [20]:
model.fc.weight.grad[0,:10]

tensor([ 8.4218e-04, -4.2140e-04, -4.9456e-04,  8.6718e-04, -7.9978e-04,
         2.8528e-04, -5.3555e-04,  5.8701e-04,  2.1346e-05, -2.7519e-04],
       device='cuda:0')

In [21]:
model.fc.weight[0,:10]

tensor([ 0.0044, -0.0150,  0.0119, -0.0344, -0.0279, -0.0348, -0.0132,  0.0086,
         0.0354,  0.0167], device='cuda:0', grad_fn=<SliceBackward0>)

In [22]:
param == model.fc.weight

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]], device='cuda:0')

In [23]:
for i, (images, target) in enumerate(train_loader):
    # measure data loading time
#     data_time.update(time.time() - end)

    if args.gpu is not None:
        images = images.cuda(args.gpu, non_blocking=True)
    if torch.cuda.is_available():
        target = target.cuda(args.gpu, non_blocking=True)

    # compute output
    output = model(images)
    loss = criterion(output, target)
    if i % 100 == 20 :
#         print(model.fc.weight.grad[0,:10])
        print(loss.item())
        print(model.fc.weight[0,:10])

    # compute gradient and do SGD step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

8.157195091247559
tensor([ 0.0049, -0.0168,  0.0115, -0.0361, -0.0297, -0.0351, -0.0120,  0.0078,
         0.0370,  0.0194], device='cuda:0', grad_fn=<SliceBackward0>)
7.018538475036621
tensor([ 0.0149, -0.0014,  0.0081, -0.0357, -0.0432, -0.0367, -0.0183,  0.0096,
         0.0436,  0.0208], device='cuda:0', grad_fn=<SliceBackward0>)
7.035290718078613
tensor([ 0.0264,  0.0027,  0.0081, -0.0365, -0.0340, -0.0374, -0.0125,  0.0053,
         0.0436,  0.0116], device='cuda:0', grad_fn=<SliceBackward0>)
7.227688312530518
tensor([ 0.0282,  0.0002,  0.0129, -0.0371, -0.0332, -0.0347, -0.0078,  0.0052,
         0.0446,  0.0121], device='cuda:0', grad_fn=<SliceBackward0>)
6.800315856933594
tensor([ 0.0278,  0.0015,  0.0186, -0.0322, -0.0360, -0.0381, -0.0030,  0.0042,
         0.0445,  0.0089], device='cuda:0', grad_fn=<SliceBackward0>)
6.87434720993042
tensor([ 0.0286, -0.0015,  0.0196, -0.0288, -0.0325, -0.0380,  0.0019,  0.0023,
         0.0424,  0.0127], device='cuda:0', grad_fn=<SliceBackw

Exception ignored in: <function WeakValueDictionary.__init__.<locals>.remove at 0x2b0b5366bd30>
Traceback (most recent call last):
  File "/home/beomgon/anaconda3/envs/pytorch/lib/python3.8/weakref.py", line 103, in remove
    def remove(wr, selfref=ref(self), _atomic_removal=_remove_dead_weakref):
KeyboardInterrupt: 


KeyboardInterrupt: 